# 9.4 Строить ассоциативные правила мы будем на основе датасета от Netflix.

In [ ]:
import pandas as pd
df = pd.read_csv('data_fin.csv', sep=';')

In [ ]:
df.info()

In [ ]:
#Из полученного датасета возьмём только те записи, у которых наивысший рейтинг (5) и объединим их по "Cust_Id". 
#Фильмы сгруппируем в строчку с разделителем "пробел" так, чтобы для каждого пользователя была строка с ID тех фильмов, которые ему понравились:
good = df[df['Rating']==5].groupby('Cust_Id')['Movie_Id'].apply(lambda r: ' '.join([str(A) for A in r]))

In [ ]:
import apyori

In [ ]:
#Теперь, когда необходимая библиотека подгружена, сделаем несколько ассоциативных правил. 
#Мы можем регулировать их количество, меняя параметры алгоритмов. Посмотрим, какие ассоциативные правила получаются для support = 0.04
association_rules = apyori.apriori(good.apply(lambda r: r.split(' ')), 
                                   min_support=0.04, 
                                   min_confidence=0.1, min_lift=2, 
                                   min_length=2)

In [1]:
association_rules

NameError: name 'association_rules' is not defined

In [ ]:
#Пройдёмся по генератору и объединим его результаты. 
asr_df = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])
for item in association_rules:
    pair = item[0] 
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] =  ' '.join(list(item[2][0][0])), \
                                  ' '.join(list(item[2][0][1])),\
                                  item[2][0][2], item[1], item[2][0][3]

    
asr_df

In [ ]:
#Для того чтобы перейти от ID фильмов к их названиям, нужно загрузить ещё один файл, в котором содержится ID фильма, год его производства и название:
titles = pd.read_csv('movie_titles.csv', encoding = "ISO-8859-1", 
                     header = None, 
                     names = ['Movie_Id', 'Year', 'Name'])

In [ ]:
#Мы можем написать процедуру, которая будет выводить названия фильмов в ассоциативном правиле и фильм, который это ассоциативное правило рекомендует:
def get_rule_title(rule):
    #9.4 в ноутбуке ошибка:
    print(titles[titles.Movie_Id.isin(rule['from'].split(' '))]['Name'].values)
    print('----------->')
    print(titles[titles.Movie_Id == int(rule['to'])]['Name'].values)
#Третий print в случае если список to из > 1 фильма не работает. Так работает:    
def get_rule_title(rule):
    print(titles[titles.Movie_Id.isin(rule['from'].split(' '))]['Name'].values)
    print('----------->')
    print(titles[titles.Movie_Id.isin(rule['to'].split(' '))]['Name'].values)

In [ ]:
get_rule_title(asr_df.loc[99])

In [ ]:
#Перейдём к построению рекомендаций для случайного человека под ID=159992. Посмотрим, какие фильмы он смотрел и как он их оценил. 
j = 159992
titles[titles.Movie_Id.isin(good.iloc[j].split(' '))]['Name']

In [ ]:
#Как мы можем посчитать рекомендации для этого человека? Мы можем пройтись по всем правилам в нашей таблице и проверить: 
#если они присутствуют в просмотрах человека и у них высокий рейтинг, значит это правило ему подходит и мы можем добавить этот фильм в список рекомендаций.
def print_rule_title(rule):
    return (titles[titles.Movie_Id == int(rule['to'])]['Name'].values)
    

result = []
for A in asr_df.index:
    if len(set(good.iloc[j].split(' ')) & set(asr_df['from'].loc[A].split(' '))) == len(asr_df['from'].loc[A].split(' ')):
        result.append(print_rule_title(asr_df.loc[A])[0])
print(set(result))

# 9.6 Продолжим работать с датасетом Netflix.

In [ ]:
#Возьмём подвыборку из 10000 случайных кастомеров и 5000 фильмов. 
cust_sample = df.Cust_Id.sample(10000)
movie_sample = df.Movie_Id.sample(5000)

In [1]:
Для генерации простых рекомендаций с помощью коллаборативной фильтрации можно воспользоваться модулем surprise. Загрузим в модуль surprise наш датасет с помощью метода Reader. 

Предварительно необходимо установить модуль surprise, он не является предустановленным.

Это можно сделать через pip. В случае, если это не работает, можно воспользоваться одним из следующих четырёх вариантов:

conda install -c conda-forge scikit-surprise
conda install -c conda-forge/label/gcc7 scikit-surprise
conda install -c conda-forge/label/cf201901 scikit-surprise
conda install -c conda-forge/label/cf202003 scikit-surprise

SyntaxError: invalid syntax (3304708402.py, line 1)

In [ ]:
import surprise
from surprise import Reader, Dataset

In [ ]:
#Возьмём только те оценки, которые относятся к выбранному подмножеству кастомеров, и только те оценки, которые относятся к выбранному подмножеству фильмов. 
#Именно в такой последовательности — сначала Cust_Id, затем Movie_Id, затем Rating.
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df[df.Cust_Id.isin(cust_sample) &
                              df.Movie_Id.isin(movie_sample)][['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [ ]:
from surprise import KNNBasic

sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)
trainingSet = data.build_full_trainset()